# WEB SCRAPING ASSIGNMENT 3

# Instructions:

    > All questions are compulsory.
    > In each of the questions you have to automate the process. You do not have to click on any button, click any clickable element, enter keywords in search boxes manually. Each process has to be performed via coding. 
    > Q1 and Q2 are connected questions i.e. after attempting Q1 proceed to Q2. Do not write whole code from beginning for Q2.
    > You may use any web scraping library and tools.
    > The question can be attempted in various ways; the correctness of question depends on the output. 
    > If you encounter any Null values during scraping, you may replace it by hyphen.

In [1]:
# Lets import all the required libraries

import pandas as pd
from bs4 import BeautifulSoup
import requests
import selenium
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from datetime import datetime

Q1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user.
For e.g. If user input is ‘guitar’. Then search for guitars.

In [2]:
#Lets load the drivers and URL
driver = webdriver.Edge("msedgedriver.exe")
driver.maximize_window()
driver.get('http://amazon.in/')
driver.implicitly_wait(5)

In [3]:
# Lets enter the details in the search column
product_vertical = input("Enter product vertical that needs to be searched:")
driver.find_element_by_id("twotabsearchtextbox").send_keys(product_vertical)
driver.find_element_by_id("nav-search-submit-button").click()

Enter product vertical that needs to be searched:Laptop


2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. 

Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", Availability", "Other Details" and “Product URL”.

In case, if any of the details are missing for any of the product then replace it by “-“.

In [5]:
#Let's create a function to get the details of all products of first three pages
urls=[]
for k in range(3):
    for i in driver.find_elements_by_xpath("//div[@class='a-section a-spacing-none']//h2"):
        urls.append(i.find_element_by_xpath(".//a").get_attribute('href'))
    try:
        next_page=driver.find_element_by_xpath("//div[@class='a-text-center']/ul/li[@class='a-last']/a").get_attribute('href')
        driver.get(next_page)
        driver.refresh()   
    except:
        break

In [6]:
# Let's create empty list to store data
brand_name=[]
name_of_the_product=[]
rating=[]
no_of_ratings=[]
price=[]
expected_delivery=[]
return_exchange=[]
other_details=[]
availability=[]
product_url=[]

In [7]:
# Let's traverse all URL one by one
for i in urls:
    driver.get(i)
    product_url.append(i)
    driver.implicitly_wait(5)
    
#Let's get the brand name
    try: brand_name.append(driver.find_element_by_id("productTitle").text.split(' ')[0]) 
    except: brand_name.append('-')
    driver.implicitly_wait(5)
    
#Let's get the name of product
    try: 
        string=''
        for i in driver.find_element_by_id("productTitle").text.split(' ')[1:]:
            string=string+' '+i
        name_of_the_product.append(string)
    except: name_of_the_product.append('-')
    driver.implicitly_wait(5)
    
#Let's scrape the rating of the product
    try: rating.append(driver.find_element_by_id("acrPopover").get_attribute('title')) 
    except: rating.append('-')
    driver.implicitly_wait(5)
    
#Let's scrape the number of ratings
    try: no_of_ratings.append(driver.find_element_by_id("acrCustomerReviewText").text) 
    except: no_of_ratings.append('-')
    driver.implicitly_wait(5)
    
#Let's scrape the price of the product
    try: 
        try: price.append(driver.find_element_by_id("priceblock_saleprice").text) 
        except:
            try: price.append(driver.find_element_by_id("priceblock_dealprice").text)
            except: price.append(driver.find_element_by_id("priceblock_ourprice").text)
    except: price.append('-')
    driver.implicitly_wait(5)
    
#Let's scrape the details of return/exchnage of the product
    try: 
        return_row=driver.find_element_by_xpath("//div[@class='a-row icon-farm-wrapper']").find_elements_by_xpath(".//div")
        found=False
        for i in return_row:
            if(i.get_attribute('data-name')=='RETURNS_POLICY'):
                found=True
                return_exchange.append(i.find_element_by_xpath(".//span[1]/div[2]/a").text)
        if(found==False):
            return_exchange.append('-')
    except: return_exchange.append('-')
    driver.implicitly_wait(5)
    
#Let's scrape the expected delivery date
    try: expected_delivery.append(driver.find_element_by_id("ddmDeliveryMessage").find_element_by_xpath(".//b").text)
    except: expected_delivery.append('-')
    driver.implicitly_wait(5)
    
#Let's scrape the availablity of the product
    try: 
        try: availability.append(driver.find_element_by_id("availability").find_element_by_xpath(".//span").text)
        except: availability.append(driver.find_element_by_id("deal_availability").find_element_by_xpath(".//div/span").text)
    except: availability.append('-')
    driver.implicitly_wait(5)
    
#Let's scrape other details
    try:
        details=[i.text.replace('\n','---') for i in driver.find_element_by_id("productDetails_techSpec_section_1").find_elements_by_xpath(".//tbody")] 
        other_details.append(details[0])
    except: other_details.append('-')
    driver.implicitly_wait(5)

In [8]:
print(len(brand_name),len(name_of_the_product),len(rating),len(no_of_ratings),len(price),len(expected_delivery),len(return_exchange),len(other_details),len(availability),len(product_url))

66 66 66 66 66 66 66 66 66 66


In [9]:
# Let's create a DataFrame for our data
amazon=pd.DataFrame({'Brand Name':brand_name,'Name of Product':name_of_the_product,'Rating':rating,'No. of Ratings':no_of_ratings,'Price':price,
             'Return/Exchange':return_exchange,'Expected Delivery':expected_delivery,'Availability':availability,'Other details':other_details,
             'Product URL':product_url})

In [10]:
#Let's save the dataframe in csv file
amazon.to_csv('products.csv')

In [13]:
amazon

,Brand Name,Name of Product,Rating,No. of Ratings,Price,Return/Exchange,Expected Delivery,Availability,Other details,Product URL
0,Lenovo,IdeaPad Flex 5 11th Gen Intel Core i7 14-inch...,4.2 out of 5 stars,17 ratings,"₹ 92,390.00",7 Days Replacement,June 9 - 10,In stock.,"Brand Lenovo---Manufacturer Lenovo, One of the...",https://www.amazon.in/gp/slredirect/picassoRed...
1,HP,Pavilion x360 10th Gen Intel Core i3 Touchscr...,4.4 out of 5 stars,458 ratings,"₹ 49,990.00",7 Days Replacement,"Tuesday, June 8",In stock.,"Brand HP---Manufacturer HP Inc., HP India Sale...",https://www.amazon.in/gp/slredirect/picassoRed...
2,HP,"245 G7 Commercial Laptop (ATHLON-3050U, 4GB R...",3.1 out of 5 stars,2 ratings,"₹ 21,999.00",7 Days Replacement,"Saturday, June 5",In stock.,Brand HP---Manufacturer HP INDIA SALES PRIVATE...,https://www.amazon.in/HP-245-1S5E8PA-ATHLON-30...
3,ASUS,VivoBook 14 (2020) Intel Core i3-1005G1 10th ...,3.7 out of 5 stars,120 ratings,"₹ 34,990.00",7 Days Replacement,"Tuesday, June 8",In stock.,"Brand ASUS---Manufacturer Asus, TECH FRONT ( C...",https://www.amazon.in/ASUS-VivoBook-i3-1005G1-...
4,HP,Chromebook 14a-na0003TU 14-inch (35.56 cms) T...,3.8 out of 5 stars,308 ratings,-,7 Days Replacement,-,,"Brand HP---Manufacturer hp, Manufactured by on...",https://www.amazon.in/HP-Chromebook-14a-na0003...
...,...,...,...,...,...,...,...,...,...,...
61,HP,14 Intel Pentium N6000 14-inch(35.6 cm) Lapto...,-,-,"₹ 31,490.00",7 Days Replacement,"Tuesday, June 8",In stock.,"Brand HP---Manufacturer HP Inc., HP India Sale...",https://www.amazon.in/HP-Pentium-14-inch-Windo...
62,Life,Digital Laptop 15.6-inch (39.62 cms) (Intel C...,3.4 out of 5 stars,16 ratings,"₹ 26,990.00",7 Days Replacement,"Saturday, June 5",In stock.,"Brand LifeDigital---Manufacturer LifeDigital, ...",https://www.amazon.in/LifeDigital-Laptop-Intel...
63,Lenovo,Ideapad S145 Intel Core i3 10th Gen 15.6” FHD...,3.0 out of 5 stars,24 ratings,"₹ 33,990.00",7 Days Replacement,"Tuesday, June 8",In stock.,"Brand Lenovo---Manufacturer LENOVO, One of the...",https://www.amazon.in/Lenovo-Ideapad-Graphics-...
64,MSI,"Modern 15, Intel 10th Gen. i5-10210U, 15.6"" (...",2.6 out of 5 stars,3 ratings,"₹ 55,990.00",7 Days Replacement,"Saturday, June 5",In stock.,"Brand MSI---Manufacturer MSI COMPUTER, Micro-S...",https://www.amazon.in/gp/slredirect/picassoRed...


In [14]:
driver.close()

3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.


In [17]:
# Lets load the URL
driver = webdriver.Edge('msedgedriver.exe')
driver.maximize_window()
url=('https://images.google.com/')
driver.get(url)
driver.implicitly_wait(2)

In [18]:
#Let's Define a function which accepts image name as an argument
def get_urls(keywords):
        driver.get(url)
        
        #Let's enter the details in the search column and click search button
        search_bar=driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
        search_bar.send_keys(keywords)
        button=driver.find_element_by_xpath("//button[@class='Tg7LZd']")
        button.click()
        driver.implicitly_wait(5)
        
        
#Lets's load few pages to scrape images from them
        for k in range(12):
            driver.find_element_by_xpath("//a[@class='wXeWr islib nfEiy mM5pbd']").send_keys(Keys.PAGE_DOWN)
        time.sleep(5)
        picture=[imgage for imgage in driver.find_elements_by_xpath("//a[@class='wXeWr islib nfEiy mM5pbd']")[:100]]
        image_urls=[]
        print(len(picture))
        if(len(picture)==100):
            for i in picture:
                try:
                    i.click()
                    image_urls.append(i.get_attribute('href'))
                except:
                    image_urls.append('Not Available')
            return image_urls
        else:
            print('Image Not Available')

In [19]:
#Let's scrape the url's of all images
fruits=get_urls('Fruits')
cars=get_urls('Cars')
machine_learning=get_urls('Machine Learning')

100
100
100


In [20]:
# Let's create a DataFrame for our data
Image_URL=pd.DataFrame({})
Image_URL['Fruits']=fruits
Image_URL['Cars']=cars
Image_URL['Machine Learning']=machine_learning
Image_URL

,Fruits,Cars,Machine Learning
0,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
1,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
2,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
3,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
4,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
...,...,...,...
95,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
96,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...
97,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=http%3A%2...
98,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...,https://www.google.com/imgres?imgurl=https%3A%...


In [21]:
driver.close()

4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. 

Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. 

Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe 
and CSV.

In [25]:
# Let's load the drivers and URL
driver=webdriver.Edge('msedgedriver.exe')
driver.maximize_window()
driver.get('https://www.flipkart.com/')
driver.implicitly_wait(3)

# Let's enter the details in the search column
product=input('Enter the product vertival that needs to be searched : ')
driver.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]').click()
driver.implicitly_wait(3)
driver.find_element_by_xpath('//input[@title="Search for products, brands and more"]').send_keys(str(product))
driver.find_element_by_xpath('//button[@type="submit"]').click()
driver.implicitly_wait(3)

Enter the product vertival that needs to be searched : smartphone


In [26]:
#Let's create an empty list to store the scrape data
product_urls=[]
brand_name=[]
smartphone_name=[]
smartphone_color=[]
RAM=[]
ROM=[]
primary_camera=[]
secondary_camera=[]
display_size=[]
display_resolution=[]
processor=[]
processor_core=[]
battery_capacity=[]
price=[]

#LEt's define a function
links=driver.find_elements_by_xpath('//a[@class="_1fQZEK"]')

#Let's scrape product URL
for i in links:
    product_urls.append(i.get_attribute('href'))
for i in product_urls:
    driver.get(i)
    driver.implicitly_wait(4)
    driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]').click()
    try:
        
#Let's scrape brand name
        title=driver.find_element_by_xpath('//span[@class="B_NuCI"]').text.split()
        brand_name.append(title[0])
    except:
        brand_name.append('-')
    try:
        
#Let's scrape smartohone name
        smartphone_name.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[1]/table/tbody/tr[3]/td[2]').text)
    except:
        smartphone_name.append('-')
    try:
        
#Let's scrape smartphone colour
        smartphone_color.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[1]/table/tbody/tr[4]/td[2]').text)
    except:
        smartphone_color.append('-')
    try:
        
#Let's scrape smartphone RAM
        RAM.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[2]/td[2]').text)
    except:
        RAM.append('-')
    try:
        
#Let's scrape smartphone ROM
        ROM.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[4]/table/tbody/tr[1]/td[2]').text)
    except:
        ROM.append('-')
    try:
        
#Let's scrape smartphone primary camera details
        primary_camera.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[2]/td[2]').text)
    except:
        primary_camera.append('-')
    try:
        
#Let's scrape smartphone secondary camera details
        secondary_camera.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[5]/table/tbody/tr[5]/td[2]').text)
    except:
        secondary_camera.append('-')
    try:
        
#Let's scrape smartphone display size
        display_size.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[1]/td[2]').text)
    except:
        display_size.append('-')
    try:
        
#Let's scrape smartphone screen resolution
        display_resolution.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[2]/table/tbody/tr[2]/td[2]').text)
    except:
        display_resolution.append('-')
    try:
        
#Let's scrape smartphone processor
        processor.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[3]/table/tbody/tr[2]/td[2]').text)
    except:
        processor.append('-')
    try:
        
#Let's scrape smartphone processor core
        processor_core.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[3]/table/tbody/tr[3]/td[2]').text)
    except:
        processor_core.append('-')
    try:
        
#Let's scrape smartphone battery capacity
        battery_capacity.append(driver.find_element_by_xpath('//div[@class="_1UhVsV"]/div[10]/table/tbody/tr[1]/td[2]').text)
    except:
        battery_capacity.append('-')
    try:
        
#Let's scrape smartphone price
        price.append(driver.find_element_by_xpath('//div[@class="dyC4hf"]/div/div/div').text)
    except:
        price.append('-')
        
        
print(len(product_urls),len(brand_name),len(smartphone_name),len(smartphone_color),len(RAM),len(ROM),len(primary_camera),len(secondary_camera),len(display_size),len(display_resolution),len(processor),len(processor_core),len(battery_capacity),len(price))

24 24 24 24 24 24 24 24 24 24 24 24 24 24


In [27]:
# Let's create a DataFrame for our data
flipkart=pd.DataFrame({})
flipkart['Brand Name']=brand_name
flipkart['Smartphone Name']=smartphone_name
flipkart['Smartphone Colour']=smartphone_color
flipkart['RAM']=RAM
flipkart['Storage (ROM)']=ROM
flipkart['Primray Camera']=primary_camera
flipkart['Secondary Camera']=secondary_camera
flipkart['Display Size']=display_size
flipkart['Display Resolution']=display_resolution
flipkart['Processor']=processor
flipkart['Processor Cores']=processor_core
flipkart['Battery Capacity']=battery_capacity
flipkart['Price']=price
flipkart['Product URL']=product_urls
flipkart

,Brand Name,Smartphone Name,Smartphone Colour,RAM,Storage (ROM),Primray Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Cores,Battery Capacity,Price,Product URL
0,Infinix,Hot 10 Play,7° Purple,4 GB,64 GB,13MP + Depth Sensor,8MP Front Camera,17.32 cm (6.82 inch),1640 x 720 Pixels,MediaTek Helio G35,Octa Core,6000 mAh,"₹8,499",https://www.flipkart.com/infinix-hot-10-play-7...
1,REDMI,9 Power,Mighty Black,4 GB,64 GB,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,"₹9,999",https://www.flipkart.com/redmi-9-power-mighty-...
2,Infinix,Hot 10 Play,Morandi Green,4 GB,64 GB,13MP + Depth Sensor,8MP Front Camera,17.32 cm (6.82 inch),1640 x 720 Pixels,MediaTek Helio G35,Octa Core,6000 mAh,"₹8,499",https://www.flipkart.com/infinix-hot-10-play-m...
3,Infinix,Hot 10 Play,Aegean Blue,4 GB,64 GB,13MP + Depth Sensor,8MP Front Camera,17.32 cm (6.82 inch),1640 x 720 Pixels,MediaTek Helio G35,Octa Core,6000 mAh,"₹8,499",https://www.flipkart.com/infinix-hot-10-play-a...
4,realme,C20,Cool Grey,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,MediaTek Helio G35,Octa Core,76.4 mm,"₹6,799",https://www.flipkart.com/realme-c20-cool-grey-...
5,realme,C20,Cool Blue,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,MediaTek Helio G35,Octa Core,76.4 mm,"₹6,799",https://www.flipkart.com/realme-c20-cool-blue-...
6,REDMI,9 Power,Electric Green,4 GB,64 GB,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,"₹9,999",https://www.flipkart.com/redmi-9-power-electri...
7,realme,Narzo 20,Glory Silver,4 GB,64 GB,48MP + 8MP + 2MP,8MP Front Camera,16.56 cm (6.52 inch),1600 x 720 Pixels,MediaTek Helio G85,Octa Core,6000 mAh,"₹10,499",https://www.flipkart.com/realme-narzo-20-glory...
8,REDMI,Note 9,Pebble Grey,4 GB,64 GB,48MP + 8MP + 2MP + 2MP,Yes,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G85,Octa Core,5020 mAh,"₹10,999",https://www.flipkart.com/redmi-note-9-pebble-g...
9,realme,Narzo 30A,Laser Blue,3 GB,32 GB,13MP + 2MP,Yes,16.54 cm (6.51 inch),1600 x 720 Pixels,MediaTek Helio G85,Octa Core,6000 mAh,"₹8,499",https://www.flipkart.com/realme-narzo-30a-lase...


In [28]:
#Let's save the dataframe in csv file
flipkart.to_csv('flipkart_smartphone.csv')

In [29]:
driver.close()

5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [30]:
# Let's load the drivers and URL
driver=webdriver.Edge('msedgedriver.exe')
driver.maximize_window()
driver.get('https://www.google.com/maps/')
driver.implicitly_wait(5)

In [31]:
#Let's create a function to enter the city name 
city = input('Enter City Name : ') 
driver.find_element_by_id('searchboxinput').send_keys(city)

Enter City Name : Jagdalpur


In [32]:
#Let's create a fucntion to search the result
driver.find_element_by_id('searchboxinput').send_keys(Keys.ENTER)
driver.implicitly_wait(6)

In [33]:
#Let's Scrape the entered city latitude and Longitude from google map
lat=driver.current_url.split('/@')[1].split(',')[0]
long=driver.current_url.split('/@')[1].split(',')[1].split(',')[0]
print('Latitude:',lat,'Longitude:',long)

Latitude: 19.0775296 Longitude: 82.0215808


In [34]:
driver.close()

6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [35]:
# Let's load the drivers and URL
driver=webdriver.Edge('msedgedriver.exe')
driver.maximize_window()
driver.get('https://trak.in/')
driver.implicitly_wait(5)

In [36]:
# Let's enter the details in the search column
search_button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(search_button)

In [37]:
#Let's create an empty list to store data
Date=[]
Startup=[]
Industry=[]
SubVertical=[]
Location=[]
Investor=[]
Investment=[]
Amount=[]

In [38]:
#Let's define a function to get the details
for i in range(48,51):
    driver.find_element_by_xpath('//div[@id="tablepress-{}_wrapper"]/div/label/select/option[4]'.format(i)).click()
    
#Let's scrape the date
    date = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in date:
        Date.append(d.text)
        
#Let's scrape the details
    startup = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in startup:
        Startup.append(n.text)
    
#Let's scrape the industry details
    industry = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in industry:
        Industry.append(n.text)
    
#Let's scrape the subvertical details
    subvertical = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in subvertical:
        SubVertical.append(s.text)

#Let's scrape the location details
    location = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in location:
        Location.append(l.text)
    
#Let's scrape the investors details
    investor = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in investor:
        Investor.append(n.text)
        
#Let's scrape the investment details
    investment = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in investment:
        Investment.append(n.text)
    
#Let's scrape the amount
    amtount = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amtount:
        Amount.append(a.text)

In [39]:
# Let's create a DataFrame for our data
trak_fund=pd.DataFrame({})
trak_fund['Date']=Date
trak_fund['Startup']=Startup 
trak_fund['Industry']=Industry
trak_fund['SubVertical']=SubVertical
trak_fund['Location']=Location
trak_fund['Investor']=Investor
trak_fund['Investment']=Investment
trak_fund['Amount']=Amount

In [40]:
trak_fund

,Date,Startup,Industry,SubVertical,Location,Investor,Investment,Amount
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


In [41]:
driver.close()

7. Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [56]:
# Let's load the drivers and URL
driver=webdriver.Edge('msedgedriver.exe')
driver.get('https://www.digit.in/')
driver.maximize_window()
driver.implicitly_wait(3)

In [57]:
top_10 = driver.find_element_by_xpath("//div[@class='menu']//ul//li[4]")
top_10.click()
driver.switch_to.window(driver.window_handles[0])

In [58]:
top_10_gam = driver.find_elements_by_xpath("//div[@class='listing_container']//ul//li")
title = input("Enter the text")

for i in top_10_gam:
    if i.text == title:
        i.click()
        
        driver.switch_to.window(driver.window_handles[0])
        
        break

Enter the textBest Gaming Laptops in India


In [59]:
urls = driver.find_elements_by_xpath("//div[@data-cat='Top_Ten_En_Scroll']//a")
pages = [i.get_attribute('href') for i in urls]

In [60]:
for i in range(len(pages)):
    driver.get(pages[i])
    driver.switch_to.window(driver.window_handles[0])
    d = [] 
    time.sleep(1)
    sidebar = driver.find_elements_by_xpath("//div[@class='tab']//a//button")
    var = []
    
    for j in range(len(sidebar)):
        sidebar[j].click()
        time.sleep(1)
        data = driver.find_elements_by_xpath("//div[@class='tabcontent']//div[{}]//div[@class='remaining-details']//tr".format(j+1))
       
        for k in data:
            var.append(k.text)
        
    x= [l.split(":") for l in var]
    cols = [l[0] for l in x]
    rows = [l[1] for l in x]
    d.append(rows)

    if i == 0:
        df = pd.DataFrame(data = d, columns = cols)
    else:
        df1 = pd.DataFrame(data = d, columns = cols)
        df = pd.concat([df,df1])

df = df.fillna('-') #fills all NaN values with '-'
df = df.reset_index().drop(columns = 'index') #resets the index and drops the column of old index

print(df)

            Model Name  Operating System (With Version)    Resolution   \
0           Area 51M R2                  Windows 10 Home   1920 x 1080   
1                m15 R3                  Windows 10 Home   3840 x 2160   
2                  ASUS                  Windows 10 Home   1920 x 1080   
3      ROG Zephyrus G14                  Windows 10 Home   1920 x 1080   
4             Legion 5i                   Windows 10 Pro   1920 x 1080   
5   ROG Zephyrus Duo 15                       Windows 10   3840 x 1100   
6       Aspire 7 Gaming                  Windows 10 Home   1920 x 1080   

  Display Size (In Inches)  Ram Included (In Gb)  Ram Type   \
0                      17.3                 16 GB      DDR4   
1                      15.6                 16 GB      DDR4   
2                      15.6                 16 GB      DDR4   
3                        14                 16 GB      DDR4   
4                      15.6                 16 GB      DDR4   
5                      15.6  

In [61]:
df

,Model Name,Operating System (With Version),Resolution,Display Size (In Inches),Ram Included (In Gb),Ram Type,Laptop Weight (In Kgs),Laptop Dimension (In Mm),Processor Model Name,Clock Speed,...,Storage Drive Capacity,Battery Type,Power Supply,Speakers,Laptop Type,Wireless Connectivity,Battery Backup (In Hours),Refresh Rate,Connectivity,Warranty Length
0,Area 51M R2,Windows 10 Home,1920 x 1080,17.3,16 GB,DDR4,4.1,27.65 x 402.6 x 319.14,10th Generation Intel® Core™ i7-10700,2.90 GHz,...,1 TB,-,-,-,-,-,-,-,-,-
1,m15 R3,Windows 10 Home,3840 x 2160,15.6,16 GB,DDR4,-,-,10th Generation Intel® Core™ i9-10980HK,-,...,1 TB,-,-,-,-,-,-,-,-,-
2,ASUS,Windows 10 Home,1920 x 1080,15.6,16 GB,DDR4,2.30,35.4 x 25.9 x 2.26,AMD Ryzen™ 9 5900HX,3.3 GHz,...,1 TB,Li-ion,240W,Yes,-,-,-,-,-,-
3,ROG Zephyrus G14,Windows 10 Home,1920 x 1080,14,16 GB,DDR4,1.65,32.5 x 22.1 x 1.8,AMD 3rd Generation Ryzen 9,3.3 GHz,...,1 TB,Lithium,180W,-,Gaming,Yes,4 Hours,-,-,-
4,Legion 5i,Windows 10 Pro,1920 x 1080,15.6,16 GB,DDR4,2.3,363.06 x 259.61 x 23.57,10th Generation Intel® Core™ i5-10300H,2.50 GHz,...,1 TB,-,-,Yes,-,-,7.7 hours,-,-,-
5,ROG Zephyrus Duo 15,Windows 10,3840 x 1100,15.6,4 GB,DDR4,2.4,268.30 x 360.00 x 20.90,Intel Core i7 10th Gen 10875H,-,...,512 GB,-,-,-,-,-,-,60 Hz,-,-
6,Aspire 7 Gaming,Windows 10 Home,1920 x 1080,15.6,8 GB,DDR4,2.15,2.29 x 36.3 x 25.4,AMD Ryzen™ 5-5500U hexa-core,-,...,512 GB,Li-ion,135 W,Yes,-,5.1,11.50 Hour,-,Yes,One-year International Travelers Warranty (ITW)


In [62]:
driver.close()